In [1]:
import os
import requests
import pandas as pd
from datetime import datetime
import json
#import mysql.connector
#from mysql.connector import Error
from dotenv import load_dotenv

In [2]:
url = "https://v3.football.api-sports.io/players?league=135&season=2023&team=496"
parameters={"league":"135","season":"2023", "team":"496", "page":"1"}
headers = {
  'x-rapidapi-key': '317673c93d4bba99406d683460d6276d',
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

response = requests.request("GET", url, headers=headers, data=parameters)

print(response.text)

{"get":"players","parameters":{"league":"135","season":"2023","team":"496"},"errors":[],"results":20,"paging":{"current":1,"total":3},"response":[{"player":{"id":31402,"name":"F. Ranocchia","firstname":"Filippo","lastname":"Ranocchia","age":22,"birth":{"date":"2001-05-14","place":"Perugia","country":"Italy"},"nationality":"Italy","height":"178 cm","weight":null,"injured":false,"photo":"https:\/\/media.api-sports.io\/football\/players\/31402.png"},"statistics":[{"team":{"id":496,"name":"Juventus","logo":"https:\/\/media.api-sports.io\/football\/teams\/496.png"},"league":{"id":135,"name":"Serie A","country":"Italy","logo":"https:\/\/media.api-sports.io\/football\/leagues\/135.png","flag":"https:\/\/media.api-sports.io\/flags\/it.svg","season":2023},"games":{"appearences":null,"lineups":null,"minutes":null,"number":null,"position":"Midfielder","rating":null,"captain":false},"substitutes":{"in":null,"out":null,"bench":null},"shots":{"total":null,"on":null},"goals":{"total":null,"conceded":

In [3]:
def get_juventus_players(url, headers, params):
    """
    Fetch the data using the API 

    """
    try:
        response = requests.request("GET", url, headers=headers, data=parameters)
        response.raise_for_status()
        return response.json()

    except requests.exceptions.HTTPError as http_error_message:
        print (f"❌ [HTTP ERROR]: {http_error_message}")

    except requests.exceptions.ConnectionError as connection_error_message:
        print (f"❌ [CONNECTION ERROR]: {connection_error_message}")

    except requests.exceptions.Timeout as timeout_error_message:
        print (f"❌ [TIMEOUT ERROR]: {timeout_error_message}")

    except requests.exceptions.RequestException as other_error_message:
        print (f"❌ [UNKNOWN ERROR]: {other_error_message}")
    
print(get_juventus_players(url=url,headers=headers, params=parameters))

{'get': 'players', 'parameters': {'league': '135', 'season': '2023', 'team': '496'}, 'errors': [], 'results': 20, 'paging': {'current': 1, 'total': 3}, 'response': [{'player': {'id': 31402, 'name': 'F. Ranocchia', 'firstname': 'Filippo', 'lastname': 'Ranocchia', 'age': 22, 'birth': {'date': '2001-05-14', 'place': 'Perugia', 'country': 'Italy'}, 'nationality': 'Italy', 'height': '178 cm', 'weight': None, 'injured': False, 'photo': 'https://media.api-sports.io/football/players/31402.png'}, 'statistics': [{'team': {'id': 496, 'name': 'Juventus', 'logo': 'https://media.api-sports.io/football/teams/496.png'}, 'league': {'id': 135, 'name': 'Serie A', 'country': 'Italy', 'logo': 'https://media.api-sports.io/football/leagues/135.png', 'flag': 'https://media.api-sports.io/flags/it.svg', 'season': 2023}, 'games': {'appearences': None, 'lineups': None, 'minutes': None, 'number': None, 'position': 'Midfielder', 'rating': None, 'captain': False}, 'substitutes': {'in': None, 'out': None, 'bench': No

In [8]:
all_responses = []
while True:
    response = requests.request("GET", url, headers=headers, params=parameters)
    if response.status_code == 200:
        data = response.json()
        all_responses.extend(data['response'])

    #to check if there are more pages within the result
        if int(data['paging']['current']) < int(data['paging']['total']):
            parameters['page'] = str(int(parameters['page']) + 1)
        else:
            break #it will break the loops once all are stored.
    else:
        print(f"[ERROR] Failed to retrieve data from API, STATUS CODE: {response.status_code}")
        print(f"Response content: {response.content}")
        break

#print combined result
print(json.dumps(all_responses, indent=5))


[
     {
          "player": {
               "id": 31402,
               "name": "F. Ranocchia",
               "firstname": "Filippo",
               "lastname": "Ranocchia",
               "age": 22,
               "birth": {
                    "date": "2001-05-14",
                    "place": "Perugia",
                    "country": "Italy"
               },
               "nationality": "Italy",
               "height": "178 cm",
               "weight": null,
               "injured": false,
               "photo": "https://media.api-sports.io/football/players/31402.png"
          },
          "statistics": [
               {
                    "team": {
                         "id": 496,
                         "name": "Juventus",
                         "logo": "https://media.api-sports.io/football/teams/496.png"
                    },
                    "league": {
                         "id": 135,
                         "name": "Serie A",
                         

In [28]:
def transformed_players(data):
    
    """"
    Parse the JSON data retrieved from API 
    """
    players = []
    for players_data in data['response']:
        statistics = players_data['statistics'][0]

        # set up constants for processing data
        player = players_data['player']
        player_id = player['id']
        player_name = player['name']
        player_age = int(player['age'])
        nationality = player['nationality']
        position = statistics['games']['position']
        rating = float(statistics['games']['rating']) if statistics['games']['rating'] else 0.0
        match_played = int(statistics['games']['appearences']) if statistics['games']['appearences'] else 0
        minutes_played = int(statistics['games']['minutes']) if statistics['games']['minutes'] else 0
        total_goals = int(statistics['goals']['total']) if statistics['goals']['total'] else 0
        total_shots = int(statistics['shots']['total']) if statistics['shots']['total'] else 0
        ontarget_shots = int(statistics['shots']['on']) if statistics['shots']['on'] else 0
        penalty_goals = int(statistics['penalty']['scored']) if statistics['penalty']['scored'] else 0
        penalty_missed = int(statistics['penalty']['missed']) if statistics['penalty']['missed'] else 0
        total_assists = int(statistics['goals']['assists']) if statistics['goals']['assists'] else 0
        total_passes = int(statistics['passes']['total']) if statistics['passes']['total'] else 0
        key_passes = int(statistics['passes']['key']) if statistics['passes']['key'] else 0
        dribble_attempts = int(statistics['dribbles']['attempts']) if statistics['dribbles']['attempts'] else 0
        dribble_success = int(statistics['dribbles']['success']) if statistics['dribbles']['success'] else 0
        tackles = int(statistics['tackles']['total']) if statistics['tackles']['total'] else 0
        interceptions = int(statistics['tackles']['interceptions']) if statistics['tackles']['interceptions'] else 0
        aerial_duel_total = int(statistics['duels']['total']) if statistics['duels']['total'] else 0
        aerial_duel_won = int(statistics['duels']['won']) if statistics['duels']['won'] else 0
        fouls = int(statistics['fouls']['committed']) if statistics['fouls']['committed'] else 0
        yellow_card = int(statistics['cards']['yellow']) if statistics['cards']['yellow'] else 0
        doubleYellowToRed = int(statistics['cards']['yellowred']) if statistics['cards']['yellowred'] else 0
        red_card = int(statistics['cards']['red']) if statistics['cards']['red'] else 0

        players.append({
            'PlayerId': player_id,
            'player': player_name,
            'age': player_age,
            'nationality': nationality,
            'position': position,
            'rating': rating,
            'match': match_played,
            'minutes': minutes_played,
            'goals': total_goals,
            'total_shots': total_shots,
            'ontarget_shots': ontarget_shots,
            'penalty_goals': penalty_goals,
            'penalty_missed': penalty_missed,
            'assists': total_assists,
            'passes': total_passes,
            'key_passes': key_passes,
            'dribble_attempts': dribble_attempts,
            'dribble_success': dribble_success,
            'tackles': tackles,
            'interception': interceptions,
            'aerial_duel_total': aerial_duel_total,
            'aerial_duel_won': aerial_duel_won,
            'fouls': fouls,
            'yellow_card': yellow_card,
            'doubleYellowToRed': doubleYellowToRed,
            'red_card': red_card
        })

    return players

In [29]:
players_list = transformed_players(data)
print(players_list)

[{'PlayerId': 286595, 'player': 'F. González', 'age': 21, 'nationality': 'Uruguay', 'position': 'Defender', 'rating': 0.0, 'match': 0, 'minutes': 0, 'goals': 0, 'total_shots': 0, 'ontarget_shots': 0, 'penalty_goals': 0, 'penalty_missed': 0, 'assists': 0, 'passes': 0, 'key_passes': 0, 'dribble_attempts': 0, 'dribble_success': 0, 'tackles': 0, 'interception': 0, 'aerial_duel_total': 0, 'aerial_duel_won': 0, 'fouls': 0, 'yellow_card': 0, 'doubleYellowToRed': 0, 'red_card': 0}, {'PlayerId': 31517, 'player': 'G. Frabotta', 'age': 25, 'nationality': 'Italy', 'position': 'Defender', 'rating': 0.0, 'match': 0, 'minutes': 0, 'goals': 0, 'total_shots': 0, 'ontarget_shots': 0, 'penalty_goals': 0, 'penalty_missed': 0, 'assists': 0, 'passes': 0, 'key_passes': 0, 'dribble_attempts': 0, 'dribble_success': 0, 'tackles': 0, 'interception': 0, 'aerial_duel_total': 0, 'aerial_duel_won': 0, 'fouls': 0, 'yellow_card': 0, 'doubleYellowToRed': 0, 'red_card': 0}, {'PlayerId': 126980, 'player': 'T. Barbieri', 